# Using EcoFOCIpy to process raw field data

## DY2208L1 (BASIS Leg I - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [26]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [27]:
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/dy2208l1/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [28]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2208L1' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2208L1.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconvert

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l1/rawconverted/ctd016.btl


In [29]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,par,scan,datetime
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8
mean,6.603237,98.184375,287.734750,32.027200,31.967213,24.944237,24.896487,6.276763,93.346375,273.508625,1.077212,7.830300,7.833275,39.910125,0.493587,19.166839,12874.250000,2022-08-18 08:33:26.124999936
min,5.618800,77.889000,244.692000,31.837800,31.832800,24.492900,24.488100,5.306100,73.560000,231.072000,-0.006600,4.938000,4.937900,2.992000,0.205300,0.283640,6672.000000,2022-08-18 08:29:08
25%,6.248800,90.173750,272.198250,31.851650,31.852875,24.570525,24.570175,5.915225,85.360500,257.667250,0.046875,6.608225,6.608300,18.028000,0.280025,0.283670,10170.000000,2022-08-18 08:31:33.500000
50%,6.650850,99.433000,289.822000,31.980700,31.889950,24.863150,24.791400,6.321700,94.516000,275.479000,0.442700,8.251250,8.252050,35.146000,0.395550,0.515315,13522.000000,2022-08-18 08:33:53.500000
75%,7.087025,108.766000,308.916500,32.202775,32.047925,25.261750,25.139850,6.762875,103.791500,294.785750,1.991000,9.487350,9.490750,56.770000,0.504200,6.054825,15841.000000,2022-08-18 08:35:29.500000
max,7.434900,115.241000,324.103000,32.274300,32.224800,25.521600,25.482300,7.088400,109.871000,309.001000,3.512600,9.944400,9.950000,88.209000,1.411400,131.930000,17776.000000,2022-08-18 08:36:50
std,0.704912,14.655986,30.829402,0.199324,0.163408,0.435575,0.406035,0.698413,14.349488,30.542054,1.327535,2.057692,2.061101,30.349610,0.388983,45.931480,3992.810566,NaN


## Create BTL report file

In [30]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd031':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [31]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2208L1',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': 'L1',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2022, 8, 18),
 'EndDate': datetime.date(2022, 8, 25),
 'Project': 'BASIS',
 'ChiefScientist': 'Alex Andrews',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'BASIS',
 'CruiseYear': 2022,
 'ctdlogs_pdf_name': 'DY22-08_COD_SHEETS.pdf'}

In [33]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54058,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2208',
 'Project_Leg': 'L1',
 'UniqueCruiseID': 'DY2208L1',
 'Project': 'BASIS',
 'StationNo_altname': 's1h2',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 55,
 'LatitudeMin': 1.32,
 'LongitudeDeg': 164,
 'LongitudeMin': 0.001,
 'GMTDay': 18,
 'GMTMonth': 'Aug',
 'GMTYear': 2022,
 'GMTTime': 59340,
 'DryBulb': -99.9,
 'RelativeHumidity': 95,
 'WetBulb': -99.9,
 'Pressure': -10000,
 'SeaState': '-999',
 'Visibility': '-99',
 'WindDir': -1000,
 'WindSpd': -99.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 92,
 'StationNameID': '',
 'MaxDepth': 90,
 'InstrumentSerialNos': '',
 'Notes': 'This is St.1 Haul 2. I had to change the name for the HABS sample because it kept giving me an error. HABS_PP works fine. COD would notlet me over-write or edit the first St.1 H.2 that I entered. Steve Porter',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7;8',
 'NutrientBtlNumbers': '1;2;3;4;5;6

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [34]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [39]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'pressure',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,pressure,turbidity,par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
6.0,6.4616,94.733,281.512,32.1033,31.9214,25.0865,24.9435,6.1275,89.835,266.957,0.1402,7.3799,7.3799,40.011,0.2261,0.34765,12694.0,2022-08-18 08:33:19,ctd001


In [40]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'pressure', 'turbidity', 'par',
       'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [41]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [42]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'pressure', 'turbidity', 'par',
       'scan', 'datetime', 'cast'],
      dtype='object')

In [44]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [45]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 5.619 5.634 6.454 ... 6.983 7.4 7.435
    oxy_percentsat_ch1  (bottle) float64 77.89 78.1 94.2 ... 106.9 114.2 115.2
    oxy_conc_ch1        (bottle) float64 244.7 245.3 281.1 ... 304.4 322.6 324.1
    salinity_ch1        (bottle) float64 32.27 32.27 32.18 ... 31.84 31.86 31.84
    salinity_ch2        (bottle) float64 32.22 32.21 31.99 ... 31.84 31.86 31.83
    sigma_t_ch1         (bottle) float64 25.52 25.52 25.18 ... 24.58 24.54 24.49
    ...                  ...
    pressure            (bottle) float64 88.21 76.31 50.26 ... 20.44 10.78 2.992
    turbidity           (bottle) float64 0.3505 1.411 0.2053 ... 0.5 0.5168
    par                 (bottle) float64 0.2837 0.2837 0.2836 ... 17.18 131.9
    scan                (bottle) float64 6.672e+03 8.415e+03 ... 1.778e+04
    datetime            (bottle) datetime64[ns] 2022-08-18T08:29:08 ... 2022-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [46]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [48]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [49]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.619 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 77.89 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 244.7 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.27 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.22 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.52 ......
    ...                  ...
    pressure            (latitude, longitude, time, bottle) float64 88.21 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.3505 .....
    par                 (latitude, longitude, time, bottle) float64 0.2837 .....
    scan                (latitude, longitude, time, bottle) float64 6.672e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [51]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.619 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 77.89 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 244.7 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.27 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.22 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.52 ......
    ...                  ...
    pressure            (latitude, longitude, time, bottle) float64 88.21 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.3505 .....
    par                 (latitude, longitude, time, bottle) float64 0.2837 .....
    scan                (latitude, longitude, time, bottle) float64 6.672e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      DY2208L1
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   L1
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-06-25T15:22:50Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

<div class="warning" style='background-color:#ffcccb; color: #FF0000; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>WARNING</b></p>
<p style='margin-left:1em;'>bottle/niskin and rosette position should be the same but can be different (example, bottles are labeled sequentially but a rosette position is skipped due to balancing or other instruments.  On this cruise the following rosette positin was fired, while the bottles where labled differently</p>
cruise_data[cast]
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"><span style="font-weight:bold">Rosette</span></th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">4</th>
    <th class="tg-0lax">5</th>
    <th class="tg-0lax">6</th>
    <th class="tg-0lax">7<br></th>
    <th class="tg-0lax">8</th>
    <th class="tg-0lax">9</th>
    <th class="tg-0lax">10</th>
    <th class="tg-0lax">11</th>
    <th class="tg-0lax">12</th>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">2</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax"><span style="font-weight:bold">Niskin</span><br></td>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">6</td>
    <td class="tg-0lax">7</td>
    <td class="tg-0lax">8</td>
    <td class="tg-0lax">9</td>
    <td class="tg-0lax">10<br></td>
    <td class="tg-0lax">11</td>
    <td class="tg-0lax">12</td>
  </tr>
</tbody>
</table>
</div>

In [52]:
import pandas as pd
nisk_btl = pd.DataFrame(data=[[3,1],[4,2],[5,3],[6,4],[7,5],[8,6],[9,7],[10,8],[11,9],[12,10],[1,11],[2,12]],columns=['rosette','bottle']) 
for cast in cruise_data.keys():
    cruise_data[cast] = pd.merge(nisk_btl, cruise_data[cast], left_on='rosette', right_on='bottle').drop('rosette',axis=1).set_index('bottle')      

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [53]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

In [54]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) int64 1 2 3 4 5 6 7 8
  * latitude            (latitude) float64 57.49
  * longitude           (longitude) float64 -164.0
  * time                (time) datetime64[ns] 2022-08-25T10:35:00
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.311 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 82.36 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 274.8 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.02 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.02 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.55 ......
    ...                  ...
    pressure            (latitude, longitude, time, bottle) float64 60.42 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.641 ......
    par                 (latitude, longitude, time, bottle) float64 0.2837 .....
    scan                (latitude, longitude, time, bottle) float64 4.054e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd016' ....
Attributes: (12/44)
    CruiseID:                      DY2208L1
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   L1
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-06-25T15:22:55Z
    date_modified:                 
    QC_indicator:                  unqcd

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history